### Importación de librerías

In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegressionCV,  LogisticRegression
from sklearn.preprocessing import StandardScaler, binarize, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score, roc_curve, auc, classification_report, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import tree
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

### Datos de Entrenamiento

In [2]:
ruta_tat = '../data/VENDROS_TAT.csv'
ruta_vendors = '../data/VENDORS_NAMES_metrics.csv'

data_tat = pd.read_csv(ruta_tat, encoding='UTF-8', sep=',')
data_vendors = pd.read_csv(ruta_vendors, encoding='UTF-8', sep=',')

C:\Users\ar037895\Anaconda3\envs\dhdsblend2022\lib\site-packages\IPython\core\interactiveshell.py:3194: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Imputación de datos

In [3]:
data_tat.dropna(how='all', axis=1)

data_tat_filter = data_tat[[
    'TAT',
    'new_PARTNO',
    'new_DESCRIPTION',
    'order_number_id',
    'order_number',
    'order_type',
    'order_date',
    'address_ship',
    'vendor_code',
    'shipment',
    'priority',
    'created_date',
    'od_confirmed_date',
    'od_req_condition']]

data_vendors_filtrado = data_vendors.drop(['COUNT','MEAN_of_TAT','STDEV_of_TAT'], axis=1)

ds_complete = data_tat_filter.merge(data_vendors_filtrado, how='left', on='vendor_code')

ds_complete.rename(columns={'new_PARTNO':'part_number','new_DESCRIPTION':'description'}, inplace=True)

ds_complete['TAT'].fillna(ds_complete.TAT.mean(), inplace=True)

ds_complete['shipment'].fillna('--', inplace=True)

ds_complete['priority'].fillna('--', inplace=True)

ds_complete['address_ship'].fillna('ARG', inplace=True)

ds_complete['od_req_condition'].fillna('CA', inplace=True)

ds_complete['order_date'] = ds_complete['order_date'].apply(lambda x: pd.to_datetime(x))
ds_complete['order_date_year'] = ds_complete['order_date'].apply(lambda x: x.year)
ds_complete['order_date_month'] = ds_complete['order_date'].apply(lambda x: x.month)
ds_complete['order_date_day'] = ds_complete['order_date'].apply(lambda x: x.day)

In [4]:
ds_complete.sample(3)

,TAT,part_number,description,order_number_id,order_number,order_type,order_date,address_ship,vendor_code,shipment,priority,created_date,od_confirmed_date,od_req_condition,vendor_name,order_date_year,order_date_month,order_date_day
7648,10.0,75111792,CALIBRE,146577,R1451120,R,2020-08-14,AR0650,AR0011,--,RTN,2020-08-14 00:00:00,2020-11-10 00:00:00,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2020,8,14
137,61.0,AMPER-750A-50MV-01,AMPERIMETRO ANALOGICO (DC),69193,R0767118,R,2018-11-23,ARG,AR0398,--,RTN,2018-11-23 00:00:00,0,CA,SICE S.R.L.,2018,11,23
4052,12.0,1243P-1,ALESAMETRO CON COMPARADOR,153630,R2037920,R,2020-12-04,AR0650,AR0011,--,RTN,2020-12-04 00:00:00,2021-01-25 00:00:00,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2020,12,4


In [5]:
ds_complete['date_arrive'] = ds_complete.apply(lambda x: x.order_date + pd.Timedelta(x.TAT, unit='D'), axis=1)

In [6]:
def  mag_identify(desc):
    if 'TORQUIMETRO'in desc or 'TORQUÍMETRO' in desc:
        desc = 'TORQUE'
    elif 'DINAM' in desc or 'CALIBRE' in desc or 'NIVEL' in desc or 'PENETRADOR' in desc or 'MICROMETRO'in desc or 'MICRÓMETRO' in desc or 'MANDRILADORA' in desc or 'SONDAS' in desc or 'DUROMETRO' in desc or 'PARALELA' in desc or 'BALANZA' in desc or 'COMPARADOR' in desc or 'CINTA' in desc or 'DUROM' in desc or 'CRIMPING' in desc or 'DUREZA' in desc or 'PLANO' in desc or 'ALESOM' in desc or 'ALESAM' in desc:
        desc = 'DIMENSIONAL'
    elif 'MANOMETRO'in desc or 'MANÓMETRO' in desc or 'VACU' in desc or 'PRESSURE' in desc:
        desc = 'PRESIÓN'
    elif 'TEMP' in desc or 'TERMO' in desc or 'HORNO' in desc or 'THERMOM' in desc:
        desc = 'TEMPERATURA'
    elif 'MEGOHM' in desc or 'ELECTRICIDAD' in desc or 'MULTIM' in desc or 'FUENTE' in desc or 'VOLTI' in desc or 'GAUSS' in desc or 'OHMMETER' in desc or 'AMPER' in desc or 'RESIS' in desc or 'CELDA' in desc or 'CRONO' in desc or 'DECADA' in desc or 'MEGOHMETRO' in desc or 'MILIOHM' in desc:
        desc = 'ELECTRICIDAD'
    elif 'BORO' in desc or 'EDDY' in desc or 'ULTRASON' in desc:
        desc = 'NDT'
    elif 'TEMPO' in desc or 'TIEMPO' in desc or 'CRONOM' in desc:
        desc = 'TIEMPO'
    elif 'FLUJO' in desc or 'FLOW' in desc:
        desc = 'FLUJO'
    elif 'PESA' in desc or 'BALANZA' in desc:
        desc = 'MASA'
    else:
        desc = 'OTROS'
    return desc

In [7]:
ds_complete['magnitud'] = ds_complete['description'].apply(lambda x: mag_identify(x))

In [8]:
ds_complete['on_time'] = ds_complete['TAT'].apply(lambda x: 1 if x <= 45 else 0)

In [9]:
ds_complete.shape

(14955, 21)

In [10]:
ds_complete

,TAT,part_number,description,order_number_id,order_number,order_type,order_date,address_ship,vendor_code,shipment,...,created_date,od_confirmed_date,od_req_condition,vendor_name,order_date_year,order_date_month,order_date_day,date_arrive,magnitud,on_time
0,12.0,879B,LCR METER,47613,R0017144TRF,R,2018-08-08,AEP,0NUU5,--,...,2018-08-31 00:00:00,0,CA,BROADVIEW INSTRUMENTATION SERV,2018,8,8,2018-08-20,OTROS,1
1,33.0,AP-2703,DINAMOMETRO,62347,R0285318,R,2018-10-03,ARG,AR0011,--,...,2018-10-03 00:00:00,0,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2018,10,3,2018-11-05,DIMENSIONAL,1
2,4.0,L10,DINAMOMETRO,67713,R0667118,R,2018-11-12,ARG,AR0011,--,...,2018-11-12 00:00:00,0,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2018,11,12,2018-11-16,DIMENSIONAL,1
3,18.0,3277705.,DINAMOMETRO,74221,R1102719,R,2019-01-07,ARG,AR0011,--,...,2019-01-05 00:00:00,0,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2019,1,7,2019-01-25,DIMENSIONAL,1
4,6.0,7034-5,COMPARADOR,76276,R1251719,R,2019-01-21,ARG,AR0011,--,...,2019-01-19 00:00:00,0,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2019,1,21,2019-01-27,DIMENSIONAL,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14950,37.0,33-215,CINTA METRICA DE 3.5 MTS/12,221104,R3023322,R,2022-11-30,AR0650,AR0011,--,...,2022-11-30 00:00:00,0,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2022,11,30,2023-01-06,DIMENSIONAL,1
14951,6.0,4000 KG,DINAMOMETRO,222035,R3095022,R,2022-12-07,AR0650,AR0011,--,...,2022-12-07 00:00:00,2023-03-09 00:00:00,CA,RUBEN HUGO COPPOLA E HIJOS SRL,2022,12,7,2022-12-13,DIMENSIONAL,1
14952,3.0,.800IN,MICROMETRO DE INTERIOR AXIAL,227645,R0259923,R,2023-01-31,AR0650,AR0002,--,...,2023-01-31 00:00:00,0,CA,MITUTOYO SUL AMERICANA LTDA.,2023,1,31,2023-02-03,DIMENSIONAL,1
14953,8.0,M22520/1-01,MANDRILADORA PINES DANIELS,229035,R0369923,R,2023-02-13,ADUANA-EZE,7VKY4,F3,...,2023-02-13 00:00:00,0,CA,AVC LABORATORY,2023,2,13,2023-02-21,DIMENSIONAL,1


### Dummies

In [11]:
features_categoricas = ['magnitud', 'order_type', 'address_ship', 'shipment', 'priority', 'vendor_name']

ds_dummies = pd.get_dummies(ds_complete[features_categoricas], drop_first=True)

In [12]:
features_numericas = ['order_date_year', 'order_date_month', 'order_date_day']

ds_final = pd.concat([ds_complete[features_numericas], ds_dummies], axis=1)
ds_final.shape

(14955, 275)

### Split

In [13]:
X = ds_final
y = ds_complete['on_time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

In [14]:
X_test

,order_date_year,order_date_month,order_date_day,magnitud_ELECTRICIDAD,magnitud_FLUJO,magnitud_MASA,magnitud_NDT,magnitud_OTROS,magnitud_PRESIÓN,magnitud_TEMPERATURA,...,vendor_name_VACUUM TECHNOLOGY INC.,"vendor_name_VALENCIA AVIATION, SL","vendor_name_VALHALLA SCIENTIFIC, INC.","vendor_name_VALUE TOOL & ENGINEERING, INC.",vendor_name_VALUETRONICS INTERNATIONAL INC,vendor_name_VASRO INC,vendor_name_VIDITEC S.A.,"vendor_name_WAYGATE TECHNOLOGIES USA, LP",vendor_name_WESCO AIRCRAFT HARDWARE CORP.,vendor_name_WIKA ARGENTINA S.A.
11246,2019,8,5,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12022,2022,5,20,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7185,2019,11,19,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9021,2018,9,4,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12791,2019,4,18,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13545,2018,10,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8240,2021,5,4,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
13063,2022,5,10,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7850,2019,3,29,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


y_trainndersample

### Pipiline comparativo

In [15]:
steps_1 = [('scaler', StandardScaler()),
         ('knn',KNeighborsClassifier())]

In [16]:
pipe_1 = Pipeline(steps_1)

In [17]:
folds = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [18]:
param_grid = {'scaler':[StandardScaler(), MinMaxScaler()],
               'knn__n_neighbors':range(2,20,2), 'knn__weights':['uniform','distance']}

grid = GridSearchCV(pipe_1, param_grid, cv=folds)

In [19]:
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__n_neighbors': range(2, 20, 2),
                         'knn__weights': ['uniform', 'distance'],
                         'scaler': [StandardScaler(), MinMaxScaler()]})

In [20]:
grid.best_params_

{'knn__n_neighbors': 18,
 'knn__weights': 'distance',
 'scaler': StandardScaler()}

In [21]:
grid.best_score_

0.7431224105822337

In [22]:
y_pred_pipe_1 = grid.best_estimator_.predict(X_test)
cm_pipe_1 =  confusion_matrix(y_test, y_pred_pipe_1)
print(cm_pipe_1)

[[1160  556]
 [ 565 2206]]


In [34]:
steps_2 = [('scaler', StandardScaler()),
         ('ensamble',RandomForestClassifier(bootstrap=True, n_jobs = -1, random_state = 127, max_samples= 0.3))]

pipe_2 = Pipeline(steps_2)

In [42]:
param_grid_pipe_2 = {'scaler':[StandardScaler(), MinMaxScaler(), None],
               'ensamble__n_estimators':range(50,100,10), 'ensamble__max_depth':range(2,10,1)}

grid_2 = GridSearchCV(pipe_2, param_grid_pipe_2, cv=folds)

In [43]:
grid_2.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('ensamble',
                                        RandomForestClassifier(max_samples=0.3,
                                                               n_jobs=-1,
                                                               random_state=127))]),
             param_grid={'ensamble__max_depth': range(2, 10),
                         'ensamble__n_estimators': range(50, 100, 10),
                         'scaler': [StandardScaler(), MinMaxScaler(), None]})

In [44]:
grid_2.best_estimator_

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('ensamble',
                 RandomForestClassifier(max_depth=9, max_samples=0.3,
                                        n_estimators=50, n_jobs=-1,
                                        random_state=127))])

In [45]:
grid_2.best_estimator_.score(X_test,y_test)

0.6422999777133942

In [46]:
grid_2.best_params_

{'ensamble__max_depth': 9,
 'ensamble__n_estimators': 50,
 'scaler': MinMaxScaler()}

In [47]:
mean_squared_error(y_test, grid_2.best_estimator_.predict(X_test))

0.35770002228660575

In [48]:
cm_pipe_2 = confusion_matrix(y_test, grid_2.best_estimator_.predict(X_test))